In [13]:
import logging
import time
import matplotlib.pyplot as plt
import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.utils import uri_helper

URI = uri_helper.uri_from_env(default='radio://0/80/2M/E7E7E7E7E7')

# Only output errors from the logging framework
logging.basicConfig(level=logging.ERROR)
######################################################################
def generateTrajectory(t, phi_1=180,phi_2=0,phi_3=180, W_max=1400):
    #traj generation as per the eqn:  in paper 
    # receives arguments based on the number of flips required
    gamma_1=phi_1/W_max
    beta_1=-3/4*(gamma_1**-3)*W_max
    gamma_3=phi_3/W_max
    beta_3=-3/4*(gamma_3**-3)*W_max
    delta_1=2*gamma_1
    delta_2_prime=delta_1+phi_2/W_max
    if(t<=delta_1):
        W=beta_1/3*((t-gamma_1)**3)-beta_1*(gamma_1**2)*t+beta_1*(gamma_1**3)/3
    elif(t>delta_1 and t<=delta_2_prime):
        W=W_max
    else:
        W=beta_3/3*((gamma_3+delta_2_prime-t)**3)-beta_3*(gamma_3**2)*(2*gamma_3+delta_2_prime-t)+beta_3*(gamma_3**3)/3
    return W

In [64]:
class FlipPlanner:
    def __init__(self, FLIP=1):
        if (FLIP==1):
            print("SINGLE FLIP, PLEASE CALCULATE")
            self.Phi_g1 = 180
            self.Phi_g2 = 0
            self.Phi_g3 = 180
            self.omega_max = 1400
            self.asc_time = 0.15
            self.duration = 0.5
        if (FLIP==2):
            print("DOUBLE FLIP, PLEASE CALCULATE")
            self.Phi_g1 = 200
            self.Phi_g2 = 300
            self.Phi_g3 = 220
            self.omega_max = 1600
            self.asc_time = 0.25
            self.duration = 0.7
        if (FLIP==3):
            print("TRIPLE FLIP, PLEASE CALCULATE")
            self.Phi_g1 = 220
            self.Phi_g2 = 520
            self.Phi_g3 = 340
            self.omega_max = 2000
            self.asc_time = 0.3
            self.duration = 0.8 

    def calculateStages(self):
        # Stage 1
        self.gamma1 = self.Phi_g1/self.omega_max
        self.beta1 = -(3/4)*self.omega_max/self.gamma1**3
        self.omega_dot_max1 = (3/4)*self.omega_max**2/self.Phi_g1
        self.delta1 = 2*self.gamma1
        # Stage 2
        self.delta2 = self.Phi_g2/self.omega_max
        # Stage 3
        self.gamma3 = self.Phi_g3/self.omega_max
        self.beta3 = -(3/4)*self.omega_max/self.gamma3**3
        self.omega_dot_max3 = (3/4)*self.omega_max**2/self.Phi_g3
        self.delta3 = 2*self.gamma3
        print("CALCULATED, RUN")

    def generateTraj2(self, t, phi_g):
        """ Generates reference angular rate trajectory 
        considering generalized flipping angle

        :param t: time since starting flipping
        :param phi_g: generalized flipping angle

        """
        omega_max = self.omega_max
        Phi_g1 = self.Phi_g1
        Phi_g2 = self.Phi_g2
        Phi_g3 = self.Phi_g3
        # Stage 1
        gamma1 = self.gamma1
        beta1 = self.beta1
        delta1 = self.delta1
        # Stage 2
        # Stage 3
        gamma3 = self.gamma3
        beta3 = self.beta3  
        delta2n = -1
        if phi_g <= Phi_g1:
            omega_d = beta1*(t - gamma1)**3/3 - beta1*gamma1**2*t + beta1*gamma1**3/3
            omega_dot_d = beta1*(t - gamma1)**2 - beta1*gamma1**2
            if t >= delta1:
                omega_d = omega_max
                omega_dot_d = 0

        if phi_g > Phi_g1 and phi_g <= (Phi_g1 + Phi_g2):
            omega_d = omega_max
            omega_dot_d = 0

        if phi_g > (Phi_g1 + Phi_g2) and phi_g <= (Phi_g1 + Phi_g2 + Phi_g3):
            if delta2n == -1:
                delta2n = t # actual end time of stage 2

            omega_d = beta3*(gamma3 + delta2n - t)**3/3 - beta3*gamma3**2*(2*gamma3 + delta2n - t) + beta3*gamma3**3/3
            omega_dot_d = -beta3*(gamma3 + delta2n - t)**2 + beta3*gamma3**2

        if phi_g > (Phi_g1 + Phi_g2 + Phi_g3):
            omega_d = 0
            omega_dot_d = 0
            
        return omega_d

    def generateTraj1(self, t):
        """ Generates reference angular rate trajectory 
        w/o considering generalized flipping angle

        :param t: time since starting flipping

        """        
        omega_max = self.omega_max
        Phi_g1 = self.Phi_g1
        Phi_g2 = self.Phi_g2
        Phi_g3 = self.Phi_g3
        # Stage 1
        gamma1 = self.gamma1
        beta1 = self.beta1
        delta1 = self.delta1
        # Stage 2
        delta2 = self.delta2
        # Stage 3
        gamma3 = self.gamma3
        beta3 = self.beta3  
        delta3 = self.delta3

        if t <= delta1:
            omega_d = beta1*(t - gamma1)**3/3 - beta1*gamma1**2*t + beta1*gamma1**3/3
            omega_dot_d = beta1*(t - gamma1)**2 - beta1*gamma1**2
            if t >= delta1:
                omega_d = omega_max
                omega_dot_d = 0

        if (delta1 < t and t <= delta1 + delta2):
            omega_d = omega_max
            omega_dot_d = 0

        if (delta1 + delta2 < t and t <= delta1 + delta2 + delta3):
            delta2n = delta1 + delta2
            omega_d = beta3*(gamma3 + delta2n - t)**3/3 - beta3*gamma3**2*(2*gamma3 + delta2n - t) + beta3*gamma3**3/3
            omega_dot_d = -beta3*(gamma3 + delta2n - t)**2 + beta3*gamma3**2

        if t > delta1 + delta2 + delta3:
            omega_d = 0
            omega_dot_d = 0
            
        return omega_d

In [ ]:
# TEST FUNCTIONS
planner = FlipPlanner(FLIP=1)
planner.calculateStages()
# print(vars(planner))
import numpy as np
t = np.linspace(0, 1)
rate = map(planner.generateTraj1, t)
# rate = map(lambda x: 2*x, list(t))
rate = np.array(list(rate))
plt.plot(t,list(rate))


In [12]:
if __name__ == '__main__':
    # Initialize the low-level drivers
    planner = FlipPlanner(FLIP=1)
    planner.calculateStages()

    cflib.crtp.init_drivers(enable_debug_driver=False)
    
    with SyncCrazyflie(URI, cf=Crazyflie(rw_cache='./cache')) as scf:
        cf = scf.cf
        h = 1   # stabilization height

        print('Initializing as PID')
        cf.param.set_value('stabilizer.controller', '1')

        cf.param.set_value('kalman.resetEstimation', '1')
        time.sleep(0.1)
        cf.param.set_value('kalman.resetEstimation', '0')
        time.sleep(2)

        print('Reset Kalman filter.')
        print('Taking off!')

        # drone take off to a height
        # TODO: redefine height
        for y in range(10):
            cf.commander.send_hover_setpoint(0, 0, 0, h*y/10)
            #cf.commander.send_stop_setpoint()
            time.sleep(0.1)

        for _ in range(50):
            cf.commander.send_hover_setpoint(0, 0, 0, h)
            #cf.commander.send_stop_setpoint()
            time.sleep(0.1)


        # execute trajectory
        # step 1: ascend
        print('Ascending!')
        asc_time = planner.start_time
        thrust = 60000  # max thrust
        for y in range(time/0.1):
            #cf.commander.send_hover_setpoint(0, 0, 0, h + 0.2 * y / 10)
            cf.commander.send_setpoint(0, 0, 0, thrust)
            time.sleep(0.1) 

        # step 2: flip (tracking desired angular rate)
        print('Flipping!')
        cf.param.set_value('stabilizationModeRoll', '0')    # to rate mode

        t_start = time.time()
        t_current = time.time()
        t = abs(t_current-t_start)  # time since starting flipping

        waypoints = []
        time_list = []
        # TODO: take flipping angle to switch trajectory
        while t <= planner.duration:
            w = planner.generateTraj(t)
            waypoints.append(w)
            time_list.append(t)
            #print(w)
            cf.commander.send_setpoint(w,0,0,thrust)
            time.sleep(0.05)    # TODO: can we leave this?
            t_current = time.time()
            t = abs(t_current-t_start)

        # step 3: stabilize
        # at a height
        print('Stabilizing!')
        cf.param.set_value('stabilizationModeRoll', '1')    # to angle mode
        for _ in range(60):
            cf.commander.send_hover_setpoint(0, 0, 0, h)
            time.sleep(0.1)

        # drone landing
        print("Landing!")
        for y in range(10):
            cf.commander.send_hover_setpoint(0, 0, 0, (10 - y)*h / 10)
            time.sleep(0.1)

        for _ in range(10):
            cf.commander.send_stop_setpoint()
            time.sleep(0.1)

        print(waypoints)
        print(time_list)
        plt.plot(time_list,waypoints)

ERROR:cflib.crazyflie:Couldn't load link driver: Cannot find a Crazyradio Dongle

Traceback (most recent call last):
  File "/home/bitcraze/.local/lib/python3.8/site-packages/cflib/drivers/crazyradio.py", line 124, in __init__
    device = _find_devices()[devid]
IndexError: list index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/bitcraze/.local/lib/python3.8/site-packages/cflib/crazyflie/__init__.py", line 231, in open_link
    self.link = cflib.crtp.get_link_driver(
  File "/home/bitcraze/.local/lib/python3.8/site-packages/cflib/crtp/__init__.py", line 99, in get_link_driver
    instance.connect(uri, link_quality_callback, link_error_callback)
  File "/home/bitcraze/.local/lib/python3.8/site-packages/cflib/crtp/radiodriver.py", line 267, in connect
    self._radio = RadioManager.open(devid)
  File "/home/bitcraze/.local/lib/python3.8/site-packages/cflib/crtp/radiodriver.py", line 226, in open
    sh

SINGLE FLIP, PLEASE CALCULATE
CALCULATED, RUN
{'Phi_g1': 180, 'Phi_g2': 0, 'Phi_g3': 180, 'omega_max': 1400, 'start_time': 0.15, 'duration': 0.5, 'gamma1': 0.12857142857142856, 'beta1': -494032.92181069974, 'omega_dot_max1': 8166.666666666667, 'delta1': 0.2571428571428571, 'delta2': 0.0, 'gamma3': 0.12857142857142856, 'beta3': -494032.92181069974, 'omega_dot_max3': 8166.666666666667, 'delta3': 0.2571428571428571}


Exception: Couldn't load link driver: Cannot find a Crazyradio Dongle

Traceback (most recent call last):
  File "/home/bitcraze/.local/lib/python3.8/site-packages/cflib/drivers/crazyradio.py", line 124, in __init__
    device = _find_devices()[devid]
IndexError: list index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/bitcraze/.local/lib/python3.8/site-packages/cflib/crazyflie/__init__.py", line 231, in open_link
    self.link = cflib.crtp.get_link_driver(
  File "/home/bitcraze/.local/lib/python3.8/site-packages/cflib/crtp/__init__.py", line 99, in get_link_driver
    instance.connect(uri, link_quality_callback, link_error_callback)
  File "/home/bitcraze/.local/lib/python3.8/site-packages/cflib/crtp/radiodriver.py", line 267, in connect
    self._radio = RadioManager.open(devid)
  File "/home/bitcraze/.local/lib/python3.8/site-packages/cflib/crtp/radiodriver.py", line 226, in open
    shared_radio 